<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pickle
import gensim
from gensim import corpora, models, similarities
from gensim.models import Word2Vec, TfidfModel, LsiModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.fasttext import FastText
#pip install glove_python

In [2]:
!pip install glove_python
from glove import Corpus, Glove# creating a corpus object

In [3]:
def data_module(name, location, output_location, dict_location, max_seq_length=100):
  dataset1 = pd.read_csv(location)
  dataset = dataset1.iloc[0:1000,:]

  if name == 'word2vec':
    data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  elif name == 'doc2vec':
    data_corpus, users_total = data_doc2vec(dataset,max_seq_length)
  elif name == 'lsi':
    data_corpus, users_total = data_lsi(dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'tfidf':
    data_corpus, users_total = data_lsi(dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'glove':
    data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  #elif name == 'fasttext':
   # data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  #data_corpus.save('/content/drive/My Drive/Movielensdata/ml25m/data_corpus')
  #print(data_corpus)
  save_object(obj=data_corpus, filename=output_location)
  #return data_corpus, users_total

def data_word2vec(dataset,max_seq_length):
  dataset.sort_values(by=['userId','timestamp'],inplace=True)
  user_total = len(dataset['userId'].unique())
  
  #Selecting the most recent movies rated by each user and padding if necessary
  movie_list = []
  for i in range(user_total):
    list1 = []
    list1 = dataset.loc[dataset['userId'] ==(i+1),['movieId']]['movieId'].tolist()
    if len(list1)>max_seq_length:
      list1 = list1[(len(list1)-max_seq_length):]
    elif len(list1)<max_seq_length:
      list1 = list1+[0 for j in range((max_seq_length-len(list1)))]
      #for j in range((max_seq_length-len(list1))):
       # list1.append(0)
    movie_list.append(list1)
  
  #Selecting the most recent ratings rated by each user and padding if necessary
  rating_list =[]
  for i in range(user_total):
    list2 = []
    list2 = dataset.loc[dataset['userId'] ==(i+1),['rating']]['rating'].tolist()
    if len(list2)>max_seq_length:
      list2 = list2[(len(list2)-max_seq_length):]
    elif len(list2)<max_seq_length:
      list2 = list2+[0 for j in range((max_seq_length-len(list2)))]
      #for j in range((max_seq_length-len(list2))):
       # list2.append(0)
    rating_list.append(list2)
  
  #Creating user_id level transpose matrices
  movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(user_total)])
  movies_transpose.index.names = ['userId']
  #print(movies_transpose)

  ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(user_total)])
  ratings_transpose.index.names = ['userId']
  #print(ratings_transpose)

  # Select features from original dataset to form a new dataframe 
  df1 = movies_transpose.iloc[:]# For each row, combine all the columns into one column
  df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
  df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
  sent = [row.split(',') for row in df_clean['clean']]

  return sent, user_total

def data_doc2vec(dataset,max_seq_length):
  Sent, user_total = data_word2vec(dataset,max_seq_length)
  tagged_data = []
  tags = []
  
  for i in range(user_total):
    tagged_data = tagged_data + [TaggedDocument(words=Sent[i], tags=[str(i)])]

  return tagged_data, user_total

def data_lsi(dataset,max_seq_length,dict_loc):
  Sent, user_total = data_word2vec(dataset,max_seq_length)
  dictionary = corpora.Dictionary(Sent)
  #print(dictionary.token2id)
  corpus = [dictionary.doc2bow(text) for text in Sent]
  dictionary.save(dict_loc)
  #corpus = np.array([[(id, freq) for id, freq in cp] for cp in corp])
  #corpus = gensim.matutils.Dense2Corpus(np.array(Sent),documents_columns=False)

  return corpus, user_total

In [4]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        pickle_object = pickle.load(input)
    return  pickle_object

In [5]:
def embedding_model(name,Data_location,model_save_location, vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 100,alpha = 0.025,min_alpha=0.00025,dm=1):
  Data = load_object(Data_location)
  #print(Data)
  if name == 'word2vec':
    word2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers)
    #voc = model1.wv
    #words = list(model1.wv.vocab)
    #vectors = model1[model1.wv.vocab]
  elif name == 'doc2vec':
    doc2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, alpha_=alpha, size_of_window=size_window, no_workers=num_workers, max_epochs=max_num_epochs,min_alpha_=min_alpha, minimum_count=mini_count, dms=dm)
  elif name == 'lsi':
    lsi(input_data=Data, save_loc=model_save_location, total_topics=topics)
  elif name == 'tfidf':
    tfidf(input_data=Data,save_loc=model_save_location)
  elif name == 'glove':
    glove_model(input_data=Data, vec_dims=vector_dims, size_of_window=size_window, save_loc=model_save_location, num_epochs=max_num_epochs, alpha_=0.05, num_threads=4)
  #elif name == 'fasttext':
   # fast_text(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers, alpha_=0.025)

def word2vec(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers):
  model = Word2Vec(input_data,min_count=minimum_count,size= vec_dims,workers=no_workers, window =size_of_window, sg = SG)
  model.save(save_loc)

def doc2vec(input_data,save_loc,vec_dims,alpha_,size_of_window,min_alpha_,minimum_count,dms,no_workers,max_epochs):
  model = Doc2Vec(size=vec_dims,
                alpha=alpha_, 
                min_alpha=min_alpha_,
                window = size_of_window,
                min_count=minimum_count,
                dm =dms)
  model.build_vocab(input_data)

  for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(input_data, total_examples=model.corpus_count, epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
  model.save(save_loc)

def lsi(input_data,save_loc,total_topics):
  model = models.LsiModel(corpus=input_data, num_topics=total_topics)
  index = similarities.MatrixSimilarity(model[input_data])
  lsi_data = model[input_data]
  lsi_topics = model.print_topics()
  #for topic in lsi_topics:
    #print(topic)
  model.save(save_loc)

def tfidf(input_data,save_loc):
  model = models.TfidfModel(corpus=input_data)
  tfidf_data = model[input_data]

  tfidf_token= np.zeros((len(tfidf_data), 350), dtype=np.float64)
  tfidf_vals= np.zeros((len(tfidf_data), 350), dtype=np.float64)
 
  for i in range(len(input_data)):
    for k in range(len(list(tfidf_data)[i])):
      tfidf_token[i][k]=(list(tfidf_data))[i][k][0]
      tfidf_vals[i][k]=(list(tfidf_data))[i][k][1]
  tfidf_list=list(tfidf_data)
  #print(list(tfidf_data))
  model.save(save_loc)

def glove_model(input_data,vec_dims,size_of_window,save_loc,alpha_=0.05,num_epochs=30, num_threads=4):
  #importing the glove library
  corpus = Corpus() #training the corpus to generate the co occurence matrix which is used in GloVe
  corpus.fit(input_data, window=size_of_window)#creating a Glove object which will use the matrix created in the above lines to create embeddings
  #We can set the learning rate as it uses Gradient Descent and number of components
  glove = Glove(no_components=vec_dims, learning_rate=alpha_) 
  glove.fit(corpus.matrix, epochs=num_epochs, no_threads=4, verbose=True)
  glove.add_dictionary(corpus.dictionary)
  glove.save(save_loc)

def fast_text(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers,alpha_=0.025):
  model = FastText(min_count=minimum_count, alpha=alpha_, size= vec_dims, workers=no_workers, window =size_of_window)
  model.build_vocab(input_data)
  model.train(input_data, epochs=model.epochs, total_examples=model.corpus_count, total_words=model.corpus_total_words)
  model.save(save_loc)

In [6]:
def scoring_module(name,model_loc,data,dict_loc):
  if name == 'word2vec':
    scores_list = score_word2vec(model_location=model_loc,input_data=data)
  elif name == 'doc2vec':
    scores_list = score_doc2vec(model_location=model_loc,input_data=data)
  elif name == 'lsi':
    scores_list = score_lsi(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'tfidf':
    scores_list = score_tfidf(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'glove':
    scores_list = score_glove(model_location=model_loc, input_data=data)
  #elif name == 'fasttext':
   # scores_list = score_fasttext(model_location=model_loc, input_data=data)
  
  return scores_list

def score_word2vec(model_location,input_data):
  model = Word2Vec.load(model_location)
  scored_data = [model.wv[text] for text in input_data]
  return scored_data

def score_doc2vec(model_location,input_data):
  model = Doc2Vec.load(model_location)
  scored_data = [model.infer_vector(text) for text in input_data]
  return scored_data

def score_glove(model_location,input_data):
  model = Glove.load(model_location)
  scored_data = [model.word_vectors[model.dictionary[text]] for doc in input_data for text in doc]
  #model.word_vectors[]
  return scored_data

def score_fasttext(model_location, input_data):
  model = FastText(model_location)
  scored_data = [model[text] for text in input_data]
  return scored_data

def score_lsi(model_location, input_data, dict_location):
  model = LsiModel.load(model_location)
  dictionary = corpora.Dictionary.load(dict_location)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  return scored_data

def score_tfidf(model_location, input_data, dict_location):
  model = TfidfModel.load(model_location)
  dictionary = corpora.Dictionary.load(dict_location)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  return scored_data

1) Name can be 'word2vec' or 'doc2vec' or 'lsi' or 'tfidf' or 'glove'.

2) location is the path to input .csv file

3) output_location is the path to save the processed data

4) dict_location is the path to dictionary of LSI or TF-IDF

Change name, output_location for all models and dict_location for LSI and TF-IDF

In [7]:
data_module(name='word2vec',max_seq_length=100,location='/content/drive/My Drive/Movielensdata/ml25m/ratings.csv',output_location='/content/drive/My Drive/Movielensdata/ml25m/word2vec/data', dict_location='/content/drive/My Drive/Movielensdata/ml25m/lsi/dict')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Change name, Data_location and model_save_location for all models

In [8]:
embedding_model(name='word2vec',Data_location='/content/drive/My Drive/Movielensdata/ml25m/word2vec/data',model_save_location='/content/drive/My Drive/Movielensdata/ml25m/word2vec/w2v', vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 10,alpha = 0.025,min_alpha=0.00025,dm=1)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
scoring_input = [['899', '2161', '3949', '5878', '1175', '1237', '8154', '2843', '7365', '4422', '6016'],['1080', '3114', '3671', '2791', '1288', '1', '541', '2692', '7323', '8014', '6370', '4703', '5147']]

Change name, model_loc or all models and dict_loc for LSI and TF-IDF 

In [10]:
scored_list = scoring_module(name='word2vec', model_loc='/content/drive/My Drive/Movielensdata/ml25m/word2vec/w2v', data=scoring_input, dict_loc='/content/drive/My Drive/Movielensdata/ml25m/lsi/dict')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
#print(scored_list[0])

In [12]:
print(scored_list)

[array([[-0.04933006,  0.04133808, -0.04917735, -0.02006493, -0.04104522,
         0.04374732,  0.04872967, -0.03336456, -0.00900195, -0.00565758],
       [ 0.02511616, -0.04690189, -0.0229129 ,  0.00749706,  0.01298854,
        -0.00767219,  0.02231734,  0.04114292,  0.00963719,  0.00451497],
       [ 0.0446034 , -0.00638184,  0.01471467, -0.02314907,  0.00109761,
        -0.03158027, -0.0307956 , -0.00399324,  0.02939923,  0.04343036],
       [-0.01658672, -0.0318711 , -0.03179268, -0.00474806,  0.00295703,
         0.01878951, -0.0005147 , -0.04795032, -0.01931167, -0.02296629],
       [ 0.03595273, -0.04801391, -0.01320856, -0.02013416, -0.04749941,
        -0.02759434,  0.02296375,  0.03000727,  0.02082018, -0.04409176],
       [-0.02858855,  0.01934585, -0.01760966,  0.00831606, -0.01414228,
         0.03060887, -0.02829752, -0.04952022,  0.03919851, -0.01542335],
       [-0.04349007, -0.04446159,  0.02109938, -0.04250612, -0.04958392,
        -0.04135284, -0.01030751,  0.0444107